<a href="https://colab.research.google.com/github/neha-duggirala/IMDB-review-classification/blob/master/IMDB_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

     |████████████████████████████████| 1.4MB 3.4MB/s 
     |████████████████████████████████| 460kB 21.3MB/s 
     |████████████████████████████████| 112kB 27.4MB/s 
     |████████████████████████████████| 102kB 10.6MB/s 
     |████████████████████████████████| 102kB 9.8MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=1e04a711d859d7116974f67ba6d0aab5c4c52fe1b8e5408ad4f09fbb802916a7
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=8cc4aef5b5069b51312775b4ed84336f1fe579befade32dd08c0e39c69506c73
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=04c

wandb: WARNING import wandb.keras called before import keras or import tensorflow.keras.  This can lead to a version mismatch, W&B now assumes tensorflow.keras


In [19]:
wandb.login()
wandb.init(project="IMDB-review",)

W&B Run: https://app.wandb.ai/neha/IMDB-review/runs/2nce4xh1

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# **In the dataset**


*   0 --> Negative review
*   1 --> Positive Review



In [2]:
imdb, info = tfds.load("imdb_reviews",with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete84G175/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete84G175/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete84G175/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
train,test=imdb['train'],imdb['test']

In [0]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []
for i,j in train:
  training_sentences.append(str(i.numpy()))
  training_labels.append(j.numpy())
  
for i,j in test:
  testing_sentences.append(str(i.numpy()))
  testing_labels.append(j.numpy())

# **Train Text PreProcessing**

In [0]:
total_word=10000
max_length = 120
oov="<OOV>"
embedding_dim = 16
num_epochs=10

In [0]:
tokenizer = Tokenizer(num_words=total_word,oov_token=oov)
tokenizer.fit_on_texts(training_sentences)

In [0]:
word_index = tokenizer.word_index

In [62]:
print("There are",max(word_index.values()),"distinct words")

There are 86539 distinct words


In [0]:
sequences = tokenizer.texts_to_sequences(training_sentences)

In [0]:
padded = pad_sequences(sequences,maxlen=max_length,truncating='post')

In [0]:
training_labels_final=np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [66]:
padded.shape

(25000, 120)

# **Test text processing**

In [0]:
test_sequence = tokenizer.texts_to_sequences(testing_sentences)
test_padded = pad_sequences(test_sequence,maxlen=max_length)

# **Model training**

In [68]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_word, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                122944    
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
Total params: 285,057
Trainable params: 285,057
Non-trainable params: 0
_________________________________________________________________


In [0]:
config = wandb.config
config.num_classes = 2
config.batch_size = 128

In [70]:
model.fit(padded, training_labels_final, 
          epochs=num_epochs, 
          validation_data=(test_padded, testing_labels_final),
          callbacks=[WandbCallback()]
          )

Epoch 1/10
  1/782 [..............................] - ETA: 0s - loss: 0.6935 - accuracy: 0.5000WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.145598). Check your callbacks.


782/782 [==============================] - 5s 7ms/step - loss: 0.4679 - accuracy: 0.7606 - val_loss: 0.3612 - val_accuracy: 0.8406
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2019 - accuracy: 0.9238 - val_loss: 0.4573 - val_accuracy: 0.8078
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0445 - accuracy: 0.9861 - val_loss: 0.7177 - val_accuracy: 0.8070
Epoch 4/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0137 - accuracy: 0.9958 - val_loss: 0.9904 - val_accuracy: 0.8059
Epoch 5/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0143 - accuracy: 0.9954 - val_loss: 0.9929 - val_accuracy: 0.8018
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0152 - accuracy: 0.9943 - val_loss: 1.0321 - val_accuracy: 0.8056
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0113 - accuracy: 0.9961 - val_loss: 1.1518 - val_accuracy: 0.8068
Epoch 8/10
782/78

In [71]:
e = model.layers[0]
weights=e.get_weights()[0]
weights.shape

(10000, 16)

In [0]:
drive='/content/drive/My Drive/datasets/NLP/IMDB'
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [0]:
import io
out_v = io.open(drive+'/vecs.tsv', 'w', encoding='utf-8')
out_m = io.open(drive+'/meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, total_word):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [74]:
decode_review(sequences[0])

"b this was an absolutely terrible movie don't be <OOV> in by christopher walken or michael <OOV> both are great actors but this must simply be their worst role in history even their great acting could not redeem this movie's ridiculous storyline this movie is an early nineties us propaganda piece the most pathetic scenes were those when the <OOV> rebels were making their cases for <OOV> maria <OOV> <OOV> appeared phony and her pseudo love affair with walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning i am disappointed that there are movies like this ruining <OOV> like christopher <OOV> good name i could barely sit through it"

In [0]:
test_Sentence = ['I love this hate  movie............wowowow best one ever']
test_Sentence_seqeunce= tokenizer.texts_to_sequences(test_Sentence)
test_Sentence_padded = pad_sequences(test_Sentence_seqeunce,maxlen=max_length,truncating='post')

In [113]:
test_Sentence_padded

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,  11, 119,  12, 797,  18,   1,
        120,  30, 127]], dtype=int32)

In [114]:
for num in test_Sentence_seqeunce[0]:
  print(reverse_word_index[num],end=" ")

i love this hate movie <OOV> best one ever 

In [115]:
model.predict(test_Sentence_padded)

array([[0.9987921]], dtype=float32)

In [116]:
model.save(drive)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: /content/drive/My Drive/datasets/NLP/IMDB/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/datasets/NLP/IMDB/assets


In [0]:

!pip install tensorflowjs
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, drive)